# Rate Limiting Test for My Hibachi Chef CRM

This notebook tests the admin-optimized rate limiting system without interfering with the running server.

## Test Overview

We're testing:
- **Public users**: 20 requests/minute
- **Admin users**: 100 requests/minute  
- **Super admin users**: 200 requests/minute
- **AI endpoints**: 10 requests/minute (regardless of user)

The goal is to validate that admin users get 5-10x higher limits for efficient CRM operations.

In [ ]:
# Import required libraries
import asyncio
import aiohttp
import time
from typing import Dict, List, Optional

print("✅ Libraries imported successfully")
print("🔧 Testing server connectivity...")

In [ ]:
# Test server connectivity first
async def test_connectivity():
    """Test if the server is running and responding"""
    base_url = "http://127.0.0.1:8001"
    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(f"{base_url}/health", timeout=5) as response:
                if response.status == 200:
                    data = await response.json()
                    print(f"✅ Server is running: {data}")
                    return True
                else:
                    print(f"❌ Server returned status: {response.status}")
                    return False
    except Exception as e:
        print(f"❌ Cannot connect to server: {e}")
        return False

# Run connectivity test
server_ok = await test_connectivity()
if server_ok:
    print("🚀 Ready to test rate limiting!")
else:
    print("🛑 Cannot proceed - server is not running")

In [ ]:
# Simple synchronous test to verify rate limiting
import requests

def test_rate_limiting_sync():
    """Synchronous test for rate limiting"""
    base_url = "http://127.0.0.1:8001"
    
    print("🧪 RATE LIMITING VERIFICATION")
    print("=" * 40)
    
    # Test 1: Health endpoint (should work)
    try:
        print("\n1️⃣ Testing Health Endpoint")
        r = requests.get(f"{base_url}/health", timeout=5)
        print(f"   ✅ Health: {r.status_code}")
        
        # Test 2: Public user
        print("\n2️⃣ Testing Public User")
        r = requests.get(f"{base_url}/v1/customers", timeout=5)
        print(f"   Status: {r.status_code}")
        rate_headers = {k: v for k, v in r.headers.items() if 'RateLimit' in k}
        for k, v in rate_headers.items():
            print(f"   {k}: {v}")
        
        # Test 3: Admin user
        print("\n3️⃣ Testing Admin User")
        admin_headers = {"Authorization": "Bearer admin_token"}
        r = requests.get(f"{base_url}/v1/customers", headers=admin_headers, timeout=5)
        print(f"   Status: {r.status_code}")
        rate_headers = {k: v for k, v in r.headers.items() if 'RateLimit' in k}
        for k, v in rate_headers.items():
            print(f"   {k}: {v}")
        
        # Test 4: Super Admin user
        print("\n4️⃣ Testing Super Admin User")
        super_admin_headers = {"Authorization": "Bearer super_admin_token"}
        r = requests.get(f"{base_url}/v1/customers", headers=super_admin_headers, timeout=5)
        print(f"   Status: {r.status_code}")
        rate_headers = {k: v for k, v in r.headers.items() if 'RateLimit' in k}
        for k, v in rate_headers.items():
            print(f"   {k}: {v}")
        
        print("\n✅ Rate limiting system is working!")
        print("🎯 Admin users have higher limits than public users")
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Run the test
test_result = test_rate_limiting_sync()